# 3. Vehicles

> Returns parent ID and registration number of vehicles which match the vehicle number passed in the argument. Ex: `0007` returns `KA53F0007` and `KA57F0007`.

## Sample request

```bash
curl 'https://bmtcmobileapi.karnataka.gov.in/WebAPI/ListVehicles' \
  -H 'Content-Type: application/json' \
  -H 'User-Agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36' \
  --data-raw '{"vehicleRegNo":"0007"}'

```


## Sample response

```json
{
    "data": [
        {
            "vehicleid": 19100,
            "vehicleregno": "KA53F0007",
            "responsecode": 200
        },
        {
            "vehicleid": 18830,
            "vehicleregno": "KA57F0007",
            "responsecode": 200
        }
    ],
    "Message": "Success",
    "Issuccess": true,
    "exception": null,
    "RowCount": 2,
    "responsecode": 200
}
```

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp bmtc.apis.vehicles

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import string
import json
import time
import datetime
from tqdm import tqdm
import geojson

import requests
import pandas as pd
from fastcore.all import Path
from traffic_data_bengaluru.utils import *
from nbdev.config import get_config

In [ ]:
#| export
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s"
)
logger = logging.getLogger(__name__)

In [ ]:
#| hide
#| eval: false

data_directory = get_data_directory() / "bmtc"

# Functions

In [ ]:
#| export
def fetch_vehicles(pattern: str = "", sleep_duration: float = 0.1):
    """Fetch vehicles matching the given registration number pattern from BMTC API."""
    time.sleep(sleep_duration)
    url = "https://bmtcmobileapi.karnataka.gov.in/WebAPI/ListVehicles"

    headers = {
        "Content-Type": "application/json",
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
        "lan": "en"
    }

    if pattern != '':
        payload = {"vehicleRegNo": pattern}
        try:
            response = requests.post(url, headers=headers, json=payload)
            response.raise_for_status()
            return response.json()['data']
        except requests.exceptions.RequestException as e:
            print(f"Error: {e}")
            return None
    else:
        vehicles = []
        characters = string.digits + string.ascii_lowercase
        for pattern in tqdm(characters, desc = 'Fetching vehicles'):
            vehicles += fetch_vehicles(pattern)
        return vehicles

In [ ]:
#| hide
#| eval: false
vehicles = fetch_vehicles(pattern='0123')
print(json.dumps(vehicles, indent=4))

In [ ]:
#| export
def process_vehicles(vehicles):
    """Process and clean vehicle data, returning a DataFrame with `vehicle_id` and `registration_number`."""
    df_vehicles = pd.DataFrame(vehicles)
    df_vehicles = df_vehicles.drop_duplicates(subset=["vehicleregno"], keep="first")

    df_vehicles.rename(columns = {'vehicleid': 'vehicle_id', 'vehicleregno': 'registration_number'}, inplace=True)
    df_vehicles = df_vehicles.sort_values(by='vehicle_id').reset_index(drop=True)
    columns = ['vehicle_id', 'registration_number']
    return df_vehicles[columns]

In [ ]:
#| hide
#| eval: false
df_vehicles = process_vehicles(vehicles)
df_vehicles

In [ ]:
# | export
def task_fetch_vehicles(data_directory: Path):
    filename = f'{str(int(datetime.datetime.now().timestamp()))}'

    logger.info("Fetching vehicles ...")
    vehicles = fetch_vehicles()

    filepath = data_directory / 'raw' / 'vehicles' / f'{filename}.json'
    filepath.parent.mkdir(parents=True, exist_ok=True)
    with open(filepath, 'w') as f:
        json.dump(vehicles, f, indent=2)
    logger.info(f"Raw vehicles saved successfully to {filepath}")

    logger.info("Processing vehicles ...")
    df_vehicles = process_vehicles(vehicles)

    filepath = data_directory / 'cleaned' / 'vehicles' / f'{filename}.csv'
    filepath.parent.mkdir(parents=True, exist_ok=True)
    df_vehicles.to_csv(filepath, index=False)
    logger.info(f"Processed vehicles saved successfully to {filepath}")

In [ ]:
#| hide
#| eval: false

task_fetch_vehicles(data_directory=data_directory)

In [ ]:
#| export
def get_vehicles(data_directory: Path):
    return process_vehicles(read_file(extract_file(get_latest_file(data_directory / 'raw' / 'vehicles'))))

In [ ]:
#| hide
#| eval: false
df_vehicles = get_vehicles(data_directory)
print(df_vehicles.shape)
df_vehicles.head()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()